# Multi Turn Conversations

In [1]:
import os
from dotenv import load_dotenv
import os
from dotenv import load_dotenv, find_dotenv
import warnings
import requests
import json
import time
# Initailize global variables
_ = load_dotenv(find_dotenv())
# warnings.filterwarnings('ignore')
url = f"{os.getenv('DLAI_TOGETHER_API_BASE', 'https://api.together.xyz')}/inference"
headers = {
        "Authorization": f"Bearer {os.getenv('TOGETHER_API_KEY')}",
        "Content-Type": "application/json"
    }


def llama(prompt, 
          add_inst=True, 
          model="togethercomputer/llama-2-7b-chat", 
          temperature=0.0, 
          max_tokens=1024,
          verbose=False,
          url=url,
          headers=headers,
          base=2, # number of seconds to wait
          max_tries=3):
    
    if add_inst:
        prompt = f"[INST]{prompt}[/INST]"

    if verbose:
        print(f"Prompt:\n{prompt}\n")
        print(f"model: {model}")

    data = {
            "model": model,
            "prompt": prompt,
            "temperature": temperature,
            "max_tokens": max_tokens
        }

    # Allow multiple attempts to call the API incase of downtime.
    # Return provided response to user after 3 failed attempts.    
    wait_seconds = [base**i for i in range(max_tries)]

    for num_tries in range(max_tries):
        try:
            response = requests.post(url, headers=headers, json=data)
            return response.json()['output']['choices'][0]['text']
        except Exception as e:
            if response.status_code != 500:
                return response.json()

            print(f"error message: {e}")
            print(f"response object: {response}")
            print(f"num_tries {num_tries}")
            print(f"Waiting {wait_seconds[num_tries]} seconds before automatically trying again.")
            time.sleep(wait_seconds[num_tries])
 
    print(f"Tried {max_tries} times to make API call to get a valid response object")
    print("Returning provided response")
    return response


def llama_chat(prompts, 
               responses,
               model="togethercomputer/llama-2-7b-chat", 
               temperature=0.0, 
               max_tokens=1024,
               verbose=False,
               url=url,
               headers=headers,
               base=2,
               max_tries=3
              ):

    prompt = get_prompt_chat(prompts,responses)

    # Allow multiple attempts to call the API incase of downtime.
    # Return provided response to user after 3 failed attempts.    
    wait_seconds = [base**i for i in range(max_tries)]

    for num_tries in range(max_tries):
        try:
            response = llama(prompt=prompt,
                             add_inst=False,
                             model=model, 
                             temperature=temperature, 
                             max_tokens=max_tokens,
                             verbose=verbose,
                             url=url,
                             headers=headers
                            )
            return response
        except Exception as e:
            if response.status_code != 500:
                return response.json()

            print(f"error message: {e}")
            print(f"response object: {response}")
            print(f"num_tries {num_tries}")
            print(f"Waiting {wait_seconds[num_tries]} seconds before automatically trying again.")
            time.sleep(wait_seconds[num_tries])
 
    print(f"Tried {max_tries} times to make API call to get a valid response object")
    print("Returning provided response")
    return response


def get_prompt_chat(prompts, responses):
  prompt_chat = f"<s>[INST] {prompts[0]} [/INST]"
  for n, response in enumerate(responses):
    prompt = prompts[n + 1]
    prompt_chat += f"\n{response}\n </s><s>[INST] \n{ prompt }\n [/INST]"

  return prompt_chat


### LLMs are stateless
LLMs don't remember your previous interactions with them by default.

In [2]:
prompt = """
    What are fun activities I can do this weekend?
"""
response = llama(prompt)
print(response)

{'id': '901e9e7e7865eb2e-SJC', 'error': {'message': 'Unable to access non-serverless model togethercomputer/llama-2-7b-chat. Please visit https://api.together.ai/models/togethercomputer/llama-2-7b-chat to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}


In [4]:
prompt_2 = """
Which of these would be good for my health?
"""
response_2 = llama(prompt_2)
print(response_2)

{'id': '901e9fd8ad6ef9f9-SJC', 'error': {'message': 'Unable to access non-serverless model togethercomputer/llama-2-7b-chat. Please visit https://api.together.ai/models/togethercomputer/llama-2-7b-chat to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}


### Constructing multi-turn prompts
You need to provide prior prompts and responses as part of the context of each new turn in the conversation.

In [5]:
prompt_1 = """
    What are fun activities I can do this weekend?
"""
response_1 = llama(prompt_1)

In [6]:
prompt_2 = """
Which of these would be good for my health?
"""

In [7]:
chat_prompt = f"""
<s>[INST] {prompt_1} [/INST]
{response_1}
</s>
<s>[INST] {prompt_2} [/INST]
"""
print(chat_prompt)


<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
{'id': '901ea019ee4df993-SJC', 'error': {'message': 'Unable to access non-serverless model togethercomputer/llama-2-7b-chat. Please visit https://api.together.ai/models/togethercomputer/llama-2-7b-chat to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}
</s>
<s>[INST] 
Which of these would be good for my health?
 [/INST]



In [ ]:
response_2 = llama(chat_prompt,
                 add_inst=False,
                 verbose=True)

In [ ]:
print(response_2)

### Use llama chat helper function

In [8]:
prompt_1 = """
    What are fun activities I can do this weekend?
"""
response_1 = llama(prompt_1)

In [9]:
prompt_2 = """
Which of these would be good for my health?
"""

In [10]:
prompts = [prompt_1,prompt_2]
responses = [response_1]

In [11]:
# Pass prompts and responses to llama_chat function
response_2 = llama_chat(prompts,responses,verbose=True)

Prompt:
<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
{'id': '901ea0893f2067ee-SJC', 'error': {'message': 'Unable to access non-serverless model togethercomputer/llama-2-7b-chat. Please visit https://api.together.ai/models/togethercomputer/llama-2-7b-chat to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}
 </s><s>[INST] 

Which of these would be good for my health?

 [/INST]

model: togethercomputer/llama-2-7b-chat


In [12]:
print(response_2)

{'id': '901ea09c4922eb2c-SJC', 'error': {'message': 'Unable to access non-serverless model togethercomputer/llama-2-7b-chat. Please visit https://api.together.ai/models/togethercomputer/llama-2-7b-chat to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}


### Try it Yourself!

In [13]:
# replace prompt_3 with your own question!
prompt_3 = "Which of these activites would be fun with friends?"
prompts = [prompt_1, prompt_2, prompt_3]
responses = [response_1, response_2]

response_3 = llama_chat(prompts, responses, verbose=True)

print(response_3)

Prompt:
<s>[INST] 
    What are fun activities I can do this weekend?
 [/INST]
{'id': '901ea0893f2067ee-SJC', 'error': {'message': 'Unable to access non-serverless model togethercomputer/llama-2-7b-chat. Please visit https://api.together.ai/models/togethercomputer/llama-2-7b-chat to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}
 </s><s>[INST] 

Which of these would be good for my health?

 [/INST]
{'id': '901ea09c4922eb2c-SJC', 'error': {'message': 'Unable to access non-serverless model togethercomputer/llama-2-7b-chat. Please visit https://api.together.ai/models/togethercomputer/llama-2-7b-chat to create and start a new dedicated endpoint for the model.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_available'}}
 </s><s>[INST] 
Which of these activites would be fun with friends?
 [/INST]

model: togethercomputer/llama-2-7b-chat
{'id': '901ea0b38a9715d8-SJC', 'error': {'me